##### Copyright 2019 The TensorFlow Authors.

This Python Notebook is created by TensorFlow, I have modified the input file and parameters 

The original file can be found here https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/text/text_generation.ipynb

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing

import numpy as np
import os
import time

In [ ]:
path_to_file = tf.keras.utils.get_file('Rap.txt', 'https://raw.githubusercontent.com/CC-94/txtfiles/main/Rap.txt')

417792/411290 [==============================] - 0s 0us/step


In [ ]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 410229 characters


In [ ]:
# Take a look at the first 250 characters in text
print(text[:250])

[Intro] Ricky Racks, I see you! Thugger! YSL for life, bitch Yeah fuck you, your momma and everything else Yung Shad, you killed this track Free the GOAT! [Chorus] That's my best friend, that's my best friend, flexin' Big ol' booty bitch missus from 


In [ ]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

101 unique characters


In [ ]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [ ]:
ids_from_chars = preprocessing.StringLookup(
    vocabulary=list(vocab), mask_token=None)

In [ ]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[58, 59, 60, 61, 62, 63, 64], [81, 82, 83]]>

In [ ]:
chars_from_ids = tf.keras.layers.experimental.preprocessing.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [ ]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [ ]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [ ]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [ ]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(410229,), dtype=int64, numpy=array([56, 38, 71, ..., 65, 62, 75])>

In [ ]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [ ]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

In [ ]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

In [ ]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'[' b'I' b'n' b't' b'r' b'o' b']' b' ' b'R' b'i' b'c' b'k' b'y' b' '
 b'R' b'a' b'c' b'k' b's' b',' b' ' b'I' b' ' b's' b'e' b'e' b' ' b'y'
 b'o' b'u' b'!' b' ' b'T' b'h' b'u' b'g' b'g' b'e' b'r' b'!' b' ' b'Y'
 b'S' b'L' b' ' b'f' b'o' b'r' b' ' b'l' b'i' b'f' b'e' b',' b' ' b'b'
 b'i' b't' b'c' b'h' b' ' b'Y' b'e' b'a' b'h' b' ' b'f' b'u' b'c' b'k'
 b' ' b'y' b'o' b'u' b',' b' ' b'y' b'o' b'u' b'r' b' ' b'm' b'o' b'm'
 b'm' b'a' b' ' b'a' b'n' b'd' b' ' b'e' b'v' b'e' b'r' b'y' b't' b'h'
 b'i' b'n' b'g'], shape=(101,), dtype=string)


In [ ]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'[Intro] Ricky Racks, I see you! Thugger! YSL for life, bitch Yeah fuck you, your momma and everything'
b" else Yung Shad, you killed this track Free the GOAT! [Chorus] That's my best friend, that's my best "
b"friend, flexin' Big ol' booty bitch missus from Texas, what's next is I'm gon' skeet off, lil' nigga "
b"come catch me, catch me And that's my bestie, my bestie, my best friend, go best friend Nigga livin' "
b"TTG and everything is still on fleek Bad bitch rollin' wit' me, she gon' smile 'cause she on fleek Hu"


In [ ]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [ ]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [ ]:
dataset = sequences.map(split_input_target)

In [ ]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'[Intro] Ricky Racks, I see you! Thugger! YSL for life, bitch Yeah fuck you, your momma and everythin'
Target: b'Intro] Ricky Racks, I see you! Thugger! YSL for life, bitch Yeah fuck you, your momma and everything'


In [ ]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [ ]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [ ]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [ ]:
model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 102) # (batch_size, sequence_length, vocab_size)


In [ ]:
model.summary()

Model: "my_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  26112     
_________________________________________________________________
gru (GRU)                    multiple                  3938304   
_________________________________________________________________
dense (Dense)                multiple                  104550    
Total params: 4,068,966
Trainable params: 4,068,966
Non-trainable params: 0
_________________________________________________________________


In [ ]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [ ]:
sampled_indices

array([ 28,  28,   5,  19,  53,  35,  10,  83,  63,  97,   4,  46,  20,
        34,  55, 101,   7,   6,  15,  21,  14,  26,  75,  33,  88,  62,
         2,  48,  57,  11,  89,  88,  82,  63,  89,  19,  88,  57,   4,
        85,  52,  59,  74,  50,  63,   2,  81,   5,  36,  68,  43,  67,
        10,  59,  77,  51,  69,  98,  88,  45,  76,  62,  76,  69,  29,
        85,  77,  85,  74,  99,  60,  98,  28,  87,   2,  27,  18,  99,
        41,  39,  44,  24,  37, 100,  55,   9,  19,  75,  69,  29,  12,
        82,  39,   6,  64,   3,  52,  79,  32,  99])

In [ ]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b"on me) Watch what you say and watch what you do, 'cause I'll slime your OG (Slime, slime) In Bali, s"

Next Char Predictions:
 b';;"2XF(zf\xe2\x80\x9d!Q3EZ\xe2\x84\xa2%$.4-9rD\xc2\xa9e\rS])\xc2\xb1\xc2\xa9yf\xc2\xb12\xc2\xa9]!\xc2\xa6WbqUf\rx"GkNj(btVl\xe2\x80\xa6\xc2\xa9Psesl?\xc2\xa6t\xc2\xa6q\xe2\x80\xb9c\xe2\x80\xa6;\xc2\xa8\r:1\xe2\x80\xb9LJO7H\xe2\x82\xacZ\'2rl?*yJ$g WvC\xe2\x80\xb9'


In [ ]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
example_batch_loss = loss(target_example_batch, example_batch_predictions)
mean_loss = example_batch_loss.numpy().mean()
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", mean_loss)

Prediction shape:  (64, 100, 102)  # (batch_size, sequence_length, vocab_size)
Mean loss:         4.6258783


In [ ]:
tf.exp(mean_loss).numpy()

102.09241

In [ ]:
model.compile(optimizer='adam', loss=loss)

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [ ]:
EPOCHS = 60

In [ ]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/60
63/63 [==============================] - 4s 42ms/step - loss: 4.0283
Epoch 2/60
63/63 [==============================] - 3s 42ms/step - loss: 2.5363
Epoch 3/60
63/63 [==============================] - 3s 42ms/step - loss: 2.2446
Epoch 4/60
63/63 [==============================] - 3s 42ms/step - loss: 2.0305
Epoch 5/60
63/63 [==============================] - 3s 42ms/step - loss: 1.8503
Epoch 6/60
63/63 [==============================] - 3s 42ms/step - loss: 1.7243
Epoch 7/60
63/63 [==============================] - 3s 42ms/step - loss: 1.6272
Epoch 8/60
63/63 [==============================] - 3s 42ms/step - loss: 1.5239
Epoch 9/60
63/63 [==============================] - 3s 42ms/step - loss: 1.4390
Epoch 10/60
63/63 [==============================] - 3s 42ms/step - loss: 1.3490
Epoch 11/60
63/63 [==============================] - 3s 42ms/step - loss: 1.2733
Epoch 12/60
63/63 [==============================] - 3s 42ms/step - loss: 1.2002
Epoch 13/60
63/63 [==================

## Generate text

In [ ]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=0.5):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [ ]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['Intro:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Intro: Birdman & (Young Thug)] That Rich Gang lifestyle Marble floors, gold to the shots (I'm calling all the shots) I'm coolin', she say she bang gangs (Gah gat) Can't you tell? (Pi'erre) I'm in this bitch with Yung Gunna, yeah, ho, YSL (YSL, what?) Walk inside the club, I smell like YSL (What? What?) Fuckin' on your thot, your bitch, she wild as hell Ooh, I spend a check up on my watch, the frame is wide as hell (Wide as hell) Ooh, fuckin' on your thot, I smell like YSL (Beep, psh-beep, thot) In this bitch with Gunna, yeah, ho, YSL (YSL, yeah) [Verse 2: Gunna] Saint Laurent (Saint Laur, "Damn) She the B-E-S-T, best (To) Prienies (Blowin' up the strip) I got the door unlocked And I know right now she'd rather have my dick than a watch (Facts) And she like it was don't comper than a wite (Ayy) Girl, I know she livin' for a nigga, dyin' for a nigga, baby? Would you die for a nigga? Hey Baby girl, would you cry for your nigga, baby? Ay, bitch, got 'em (Uh hundred chopser) When I drop tha